In [31]:
#Import Statements
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time

In [3]:


#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
#from selenium import webdriver
#firefox_binary = FirefoxBinary()
#driver = webdriver.Firefox(firefox_binary=firefox_binary)

#driver = webdriver.Firefox()
#driver.get("http://www.python.org")
#assert "Python" in driver.title
#elem = driver.find_element(By.NAME, "q")
#elem.clear()
#elem.send_keys("pycon")
#elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
#driver.close()

In [47]:
#Load Rochelle Data
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")
scraped_data_frame = pd.DataFrame()
#rochelle.head


In [44]:
#rochelle.columns

In [28]:
#rochelle.reset_index(inplace = True,drop = True) 
#name = rochelle.loc[2]["Scientific Name"] #for some reason couldn't access rochelle[0]...
#print(name)


Phragmites australis


In [7]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [8]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
# url = "https://quod.lib.umich.edu/h/herb00ic/x-1000138/MICH-V-1000138A?lasttype=boolean;lastview=reslist;op2=And;q2=Phragmites+australis+australis;resnum=1;rgn2=ic_all;select2=all;size=50;start=1;subview=detail;view=entry"
#html = urllib.request.urlopen(url, context=ctx).read()
#soup = BeautifulSoup(html, 'html.parser')
#soup

In [9]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
#resp = requests.get(url)
#newsoup = BeautifulSoup(resp.content, 'html.parser')
#iteminfo = soup.find('dl')
#print(iteminfo)

In [10]:
#accessing search page of the herbarium
herbarium_url = "https://quod.lib.umich.edu/cgi/i/image/image-idx?xc=1;page=searchgroup;g=herb-ic"
herbarium_html = urllib.request.urlopen(herbarium_url, context=ctx).read()
#herbarium_html


In [79]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?

#send driver to the herbarium website and submit a search query and scrape the amount of results
#BE SURE TO SWITCH ALL "FIND ELEMENTS" TO XPATH IF PLAUSIBLE?
driver.get(herbarium_url)
elem = driver.find_element(by = By.NAME, value = 'q1') #Q1 is the first search box?
elem.send_keys('Cabomba caroliniana') #Sends in a name from Rochelle's list
searchbutton = driver.find_element(by = By.CSS_SELECTOR, value= '#collection-search > button.\[.button.button--cta.\]') #selects the advanced search button based off the pulled tag from right clicking the inspected element
searchbutton.click()
#search_results_url = driver.current_url
resultcount = driver.find_element(by = By.CLASS_NAME, value ='results-heading').text
#next line returns a list of two numbers like ['23','23']
resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results
resultitem = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/section[1]/a')
resultitem.click()



'''
if resultcountnumberlist[1]<= 50:
    for result in range(int(resultcountnumberlist[0])): #properly traverses the amount of results on the page
        #print(result)
        #resultitem = driver.find_element(by = By.CSS_SELECTOR, value='#maincontent > div.\[.flex.flex-flow-rw.\]\[.flex-gap-1.\] > div.main-panel > section:nth-child(4) > a')
        resultitem = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/section['+str(result+1) +']/a')
        resultitem.click()
        #driver.switch_to.window(driver.window_handles[1])
        #driver.get(search_results_url)

        ####scrape data
        ####return back to results
elif resultcountnumberlist[1]>50:
    print('There is More than One Page of Results')
    

#individually select each search result and scrape each matching entry'''

"\nif resultcountnumberlist[1]<= 50:\n    for result in range(int(resultcountnumberlist[0])): #properly traverses the amount of results on the page\n        #print(result)\n        #resultitem = driver.find_element(by = By.CSS_SELECTOR, value='#maincontent > div.\\[.flex.flex-flow-rw.\\]\\[.flex-gap-1.\\] > div.main-panel > section:nth-child(4) > a')\n        resultitem = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/section['+str(result+1) +']/a')\n        resultitem.click()\n        #driver.switch_to.window(driver.window_handles[1])\n        #driver.get(search_results_url)\n\n        ####scrape data\n        ####return back to results\nelif resultcountnumberlist[1]>50:\n    print('There is More than One Page of Results')\n    \n\n#individually select each search result and scrape each matching entry"

In [107]:
#driver2 = webdriver.Chrome()
for result in range(int(resultcountnumberlist[0]))[:1]: #[:-1]: #properly traverses the amount of results on the page
    time.sleep(2)
    resp = requests.get(driver.current_url)
    #print(resp.content)
    newsoup = BeautifulSoup(resp.content, 'html.parser')
    collection_category = newsoup.find_all('dt')
    collection_data = newsoup.find_all('dd')

    nextbutton = driver.find_element(by=By.XPATH, value='/html/body/main/div[1]/p/a[2]')
    nextbutton.click()

In [111]:
print(collection_category)

[<dt data-key="catalognumber">Catalog Number</dt>, <dt data-key="scientificname">Scientific Name</dt>, <dt data-key="family">Family</dt>, <dt data-key="recordedby">Collector</dt>, <dt data-key="recordnumber">Collector Number</dt>, <dt data-key="eventdate">Event Date</dt>, <dt data-key="continent">Continent</dt>, <dt data-key="country">Country</dt>, <dt data-key="stateprovince">State/Province</dt>, <dt data-key="county">County</dt>, <dt data-key="locality">Locality</dt>, <dt data-key="collectioncode">Collection Code</dt>, <dt data-key="institutioncode">Institution Code</dt>, <dt data-key="herb_modified">Modified</dt>, <dt data-key="id">ID</dt>, <dt data-key="occurrenceid">Occurrence ID</dt>, <dt data-key="collection">Collection</dt>, <dt data-key="image_size">Image Size</dt>, <dt data-key="reported_size">File Size</dt>, <dt data-key="m_id">Record</dt>, <dt data-key="bookmark">Link to this Item</dt>, <dt>Manifest</dt>, <dt>Full citation</dt>]


TypeError: list indices must be integers or slices, not str